In [26]:
from counter import doc_iterator

In [27]:
#jstor_zip_base = "G:/My Drive/2020 ORGANISATION/1. PROJECTS/qualitative analysis of literature/110 CITATION ANALYSIS/000 data/sociology jstor"
jstor_zip_base = "Z:/google_drive/1. PROJECTS/qualitative analysis of literature/110 CITATION ANALYSIS/000 data/sociology jstor"
#jstor_zip_base = "C:/Users/amcga/Desktop/jstor_soc"

In [28]:
def get_di():
    return doc_iterator(
        jstor_zip_base,
        SKIP_N=99, # takes 1/100 documents
    )

In [29]:
import regex as re
regexp = r'[\w\'`]+'

In [30]:
def text_iterator(it):
    for i,(doi, doc) in enumerate(it):
        if i%1000 == 0:
            print(f"Document {i}")
            
        yield re.findall( regexp, doc['content'] )

from gensim import corpora

di = get_di()
dictionary = corpora.Dictionary( text_iterator(di) )

Iterating over  248380 documents


In [31]:
import gensim

In [34]:
class MyCorpus(object):
    def __init__(self):
        self.di = get_di()        
        self.ti = text_iterator(self.di)
    def __iter__(self):
        yield dictionary.doc2bow( next(self.ti) )

corpus = MyCorpus()
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                      id2word=dictionary,
                                      num_topics=100,
                                      update_every=10,
                                      chunksize=10000,
                                      passes=10)

Iterating over  248380 documents


In [35]:
lda.print_topics()

[(16,
  '0.000*"the" + 0.000*"of" + 0.000*"and" + 0.000*"in" + 0.000*"to" + 0.000*"a" + 0.000*"on" + 0.000*"Ghana" + 0.000*"that" + 0.000*"clergy"'),
 (0,
  '0.000*"ft4" + 0.000*"gal" + 0.000*"explorative" + 0.000*"fendant" + 0.000*"fluenced" + 0.000*"forcement" + 0.000*"fringement" + 0.000*"evaluat" + 0.000*"haystack" + 0.000*"howcess"'),
 (59,
  '0.000*"the" + 0.000*"of" + 0.000*"in" + 0.000*"and" + 0.000*"to" + 0.000*"political" + 0.000*"on" + 0.000*"clergy" + 0.000*"The" + 0.000*"that"'),
 (48,
  '0.000*"the" + 0.000*"of" + 0.000*"and" + 0.000*"in" + 0.000*"to" + 0.000*"The" + 0.000*"on" + 0.000*"political" + 0.000*"Ghana" + 0.000*"a"'),
 (19,
  '0.000*"the" + 0.000*"of" + 0.000*"and" + 0.000*"in" + 0.000*"to" + 0.000*"The" + 0.000*"political" + 0.000*"a" + 0.000*"on" + 0.000*"that"'),
 (89,
  '0.000*"the" + 0.000*"of" + 0.000*"and" + 0.000*"in" + 0.000*"to" + 0.000*"The" + 0.000*"on" + 0.000*"political" + 0.000*"a" + 0.000*"that"'),
 (97,
  '0.000*"the" + 0.000*"of" + 0.000*"and" 